In [1]:
import sys
import time
import json
import pandas as pd
sys.path.append("..")
from utils.tool import (
    # load_config,
    load_csv,
    save_csv,
    parse_timestamp,
    build_releases_windows,
    build_months_windows,
    load_list_via_txt,
)
community="tensorflow"

In [25]:
commits_pytorch = pd.read_csv(f'../../data/raw/commits/{community}.csv')
commits_pytorch

,hash,timestamp,name,email,LOC
0,d3651585b84c5c871d221f777b8692d96acbd292,2018-06-19T02:06:50Z,Marat Dukhan,maratek@gmail.com,61.0
1,fc3555ce4d89384e0ba4b87da22c2881c4270d13,2022-07-22T22:26:09Z,goldenxuett,ttanx@fb.com,153.0
2,ca500f32dac25da237aab3761b28405a30517f9e,2022-03-22T18:57:26Z,Nikita Shulga,nshulga@fb.com,13.0
3,4b00bce156869ba71475aa3fff80eec553e9da67,2021-03-16T22:38:17Z,Yi Wang,wayi@fb.com,10.0
4,c6a7b94ae79bd35879f7a4f509723ae350519aa0,2022-01-06T17:27:26Z,Richard Zou,zou3519@users.noreply.github.com,206.0
...,...,...,...,...,...
48349,224975142274fd8d3a8dee5feb7a08f034408cae,2018-07-17T20:58:17Z,Peter Goldsborough,psag@fb.com,151.0
48350,61dd35f1d6489b4e4a5c584c2f3bf5f161758a4e,2017-03-15T17:56:36Z,Kittipat Virochsiri,kittipat@fb.com,151.0
48351,9f519d2d2d5dd4490de43ec2bcea59efab13e225,2021-04-09T21:47:24Z,Sam Estep,sestep@fb.com,3.0
48352,02df1ccd9c161cc42f15aac0fafb648a86c7834a,2019-05-10T01:03:41Z,Owen Anderson,owen.anderson@oculus.com,51.0


In [22]:
issues = pd.read_csv(f"../../data/raw/issues/{community}.csv")
issues

,issue_id,author,issue_type,timestamp,title,body
0,1,mrry,issue,2015-11-09T14:21:11Z,Add support for Python 3.x,"Currently we only support Python 2.7, but we s..."
1,2,ZhengRui,issue,2015-11-09T15:00:48Z,"OSX Yosemite ""can't determine number of CPU co...",``` python\r\n>>> import tensorflow as tf\r\n>...
2,3,sirinath,issue,2015-11-09T15:11:28Z,"JVM, .NET Language Support","Your website says ""... contribute SWIG interfa..."
3,4,panmari,issue,2015-11-09T15:20:48Z,Installation over pip fails to import with pro...,Pip freeze reports the following protobuf vers...
4,5,ravwojdyla,issue,2015-11-09T16:18:36Z,Java interface,Issue to trace effort of swig interface for ja...
...,...,...,...,...,...,...
56555,56961,ghost,issue,2022-07-31T11:10:58Z,Wrong gradient in TensorScatterUpdate with dup...,<details><summary>Click to expand!</summary> \...
56556,56962,Blackbird95x,issue,2022-07-31T20:42:34Z,INVALID_ARGUMENT: assertion failed: [predicti...,<details><summary>Click to expand!</summary> \...
56557,56963,hans-ekbrand,issue,2022-07-31T21:53:17Z,build failure on Debian,<details><summary>Click to expand!</summary> \...
56558,59505,Ethylbenzol,issue,2021-07-16T13:34:09Z,"Sucessful tflite conversion to uint8, but mode...",# Prerequisites\r\n\r\nPlease answer the follo...


In [23]:
issue_event = pd.read_csv(f'../../data/raw/events/{community}.csv')
issue_event

,issue_id,actor,timestamp,type,body
0,1,girving,2015-11-09T14:24:41Z,commented,"Main things this involves: `print -> print()`,..."
1,1,cdnsteve,2015-11-09T15:01:33Z,commented,:+1: to this issue\n
2,1,dvbuntu,2015-11-09T15:30:56Z,commented,:+1:\n
3,1,ashwin31,2015-11-09T16:36:05Z,commented,:+1:\n
4,1,girving,2015-11-09T16:54:58Z,assigned,NaN
...,...,...,...,...,...
1184272,59516,ravikyram,2020-07-03T04:31:12Z,assigned,NaN
1184273,59516,ravikyram,2020-07-03T04:31:20Z,unlabeled,NaN
1184274,59516,saberkun,2020-07-03T19:21:20Z,commented,DALI is not part of first-party tensorflow pac...
1184275,59516,bhack,2020-07-12T11:04:56Z,commented,We need to find a solution for AMD RALI https:...


In [26]:
issue_author = set(issues['author'].unique().tolist())
issue_event_actor = set(issue_event['actor'].unique().tolist())

In [28]:
len(issue_author.union(issue_event_actor))

44664

In [15]:
len(issue_event['actor'].unique())

42498

In [15]:
star_data_tensorflow = pd.read_csv('../../data/raw/stars/tensorflow_releases.csv')
star_data_tensorflow.drop(0, inplace=True)
star_data_tensorflow

,timestamp,stars
1,2016.02.16,16140
2,2016.06.27,23975
3,2016.11.09,32660
4,2017.02.15,40934
5,2017.06.15,54484
6,2017.11.02,68113
7,2018.02.28,82321
8,2018.07.10,95862
9,2018.11.05,104869
10,2019.06.19,121121


In [5]:
for i, (save_name, window, date) in enumerate(zip(labels, time_windows, dates)):
    print(save_name, window)

0.5.0 ['2015.11.09', '2016.02.16']
0.6.0 ['2016.02.16', '2016.06.27']
0.9.0 ['2016.06.27', '2016.11.09']
0.11.0 ['2016.11.09', '2017.02.15']
1.0.0 ['2017.02.15', '2017.06.15']
1.2.0 ['2017.06.15', '2017.11.02']
1.4.0 ['2017.11.02', '2018.02.28']
1.6.0 ['2018.02.28', '2018.07.10']
1.9.0 ['2018.07.10', '2018.11.05']
1.12.0 ['2018.11.05', '2019.06.19']
1.14.0 ['2019.06.19', '2019.09.30']
2.0.0 ['2019.09.30', '2020.01.08']
2.1.0 ['2020.01.08', '2020.05.06']
2.2.0 ['2020.05.06', '2020.07.27']
2.3.0 ['2020.07.27', '2020.12.14']
2.4.0 ['2020.12.14', '2021.05.13']
2.5.0 ['2021.05.13', '2021.08.11']
2.6.0 ['2021.08.11', '2021.11.05']
2.7.0 ['2021.11.05', '2022.02.02']
2.8.0 ['2022.02.02', '2022.05.16']


In [8]:
release

NameError: name 'release' is not defined

In [7]:
labels = build_releases_windows_test(f'../../data/raw/releases_time/{community}.csv')

KeyError: 21

In [ ]:
dates

In [ ]:
events_file=f"../../data/raw/events/{community}.csv"
events = load_csv(events_file)
events

In [ ]:
len(events['actor'].unique())

In [ ]:
commits = pd.read_csv(f"../../data/raw/commits/{community}.csv")
commits

In [ ]:
len(commits['name'].unique())

In [ ]:
len(issues[issues['author'] == author_name])

In [2]:
developers_file=f"../../data/preprocessing/{community}_developers.csv"
commits_file=f"../../data/raw/commits/{community}.csv"
issues_file=f"../../data/raw/issues/{community}.csv"
comments_file=f"../../data/raw/events/{community}.csv"
removed_authors=load_list_via_txt(f"../../data/raw/bots/{community}_authors.txt")
removed_logins=load_csv(f"../../data/raw/bots/{community}_logins.csv")["login"]

In [3]:
developers_pytorch = pd.read_csv(developers_file)
developers_pytorch['login'].unique()

array(['001ca087c6ea0583345939d7f4ed860d',
       '015dac449cb92d4dad88134c53621017',
       '07ebcafe6857f5261d097933dfeb351e', ..., 'zxjzxj9', 'zyeric',
       'zzpmiracle'], dtype=object)

In [4]:
contribution_table_pytorch = pd.read_csv('../../data/contributions/global/tensorflow.csv')
contribution_table_pytorch


,login,commit_count,LOC,issue_count,central_degree
0,chr1sj0nes,159,43980,0,82
1,benoitsteiner,1550,644677,102,44515
2,yuanbopeng,21,344,17,2107
3,teijeong,234,27640,0,5167
4,ckkuang,107,14337,1,977
...,...,...,...,...,...
23033,tgeffroyModuleus,0,0,1,0
23034,Dclpwos,0,0,1,0
23035,Piecer-plc,0,0,1,0
23036,Blackbird95x,0,0,1,0


In [5]:
contribution_table_pytorch[contribution_table_pytorch['commit_count'] > 0]['login'].unique()


array(['chr1sj0nes', 'benoitsteiner', 'yuanbopeng', ..., 'joapaspe',
       'agrawalnishant', 'kpraving'], dtype=object)

In [6]:
list_pytorch_developers = developers_pytorch['login'].unique().tolist()
list_pytorch_contributions = contribution_table_pytorch[contribution_table_pytorch['commit_count'] > 0]['login'].unique()
# difference of the two list


In [7]:
len(list_pytorch_contributions)

3288

In [8]:
len(list_pytorch_developers)

3290

In [9]:
# list differnce
list(set(list_pytorch_developers) - set(list_pytorch_contributions))

['958ce6efbab88b5f55718be7ef42850e',
 '137deaad96c3b8ef220cc5bd701d31b7',
 'cbaf0bdc46aec92c730fbcf727a85c3d',
 '4fa4ac52e4b9abfd7f32434e5d8ef6bf',
 'c8b0ce6ff038ab19d46d6873159f21db',
 'c2ec6c31e315e2af9521706e7bcfcc8d',
 'b7f976f1b9089642eab08239ec9af7fa',
 '62787a44fe6b3ee83757d6196c53f343',
 '08f59abfa1b3ec5ee802899865e6dac2',
 '164298aef72051c4144544f8e040620a',
 '8b023a0c10703b7c81379243c963169b',
 '0c22164b7ba44189cafa06c9a4d3191a',
 'f5d1eac046964aff5294d9b8aefe176f',
 '41fbc7e4946cd7bcc079658e52387619',
 'ff68e5aecd9ac4fd1646fb6355a22af7',
 '64bd7695f706952bf3d989546e4849f6',
 '6277cb4d8a086584131a466d2016914b',
 'c7823fe270b68c71432481e5e417dc1b',
 '338704e621817cb86e73ed5d3675ea80',
 '5bb9d62e05f6742656174f712265af66',
 '97d97e872bed83fe5015de9d51b023d5',
 'f635a24ccb3db1001b09509760d30168',
 'e1e49a15e2e825643ba3cff23d36ea6a',
 'e3ab7bd6981f81d35bc96853e5cb6ea2',
 '9ce74c837804c9256831aba3ac4cf0c8',
 '1e9193e5e2ed4965346232f258caea21',
 '5ddd45d11fd73e9661e289d8c89c4078',
 

In [ ]:
contribution_table_pytorch = pd.read_csv(f"../../data/contributions/global/pytorch.csv")
sorted(contribution_table_pytorch['issue_count'], reverse=True)


In [ ]:
commits = load_csv(commits_file)
issues = load_csv(issues_file)
comments = load_csv(comments_file)
issues

In [ ]:
issues = issues[issues["author"].isin(removed_authors)==False]
issues = issues[issues["author"].isin(removed_logins)==False]
issues